In [3]:

import pandas as pd
import io

df = pd.read_excel("/Users/dhui/Downloads/Test set values.xlsx",engine='openpyxl')
pd.options.display.max_columns = None


In [4]:
#Change all unknown permit to false
df['permit'].replace(['Unknown'],'False')

0        1.0
1        1.0
2        NaN
3        1.0
4        1.0
        ... 
14845    1.0
14846    0.0
14847    NaN
14848    1.0
14849    1.0
Name: permit, Length: 14850, dtype: float64

In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
#Only want to cluster on a few of these
sdf = df.drop(columns=['amount_tsh','date_recorded','longitude','latitude','recorded_by','construction_year','extraction_type_group','extraction_type_class','management_group','payment_type','quantity_group','source_type','source_class','waterpoint_type_group','num_private'])

def get_col_names(sdf):
  n_features = sdf.shape[1]

  cat_col_names = list(sdf.select_dtypes(include=np.object).columns)
  num_col_names = list(sdf.select_dtypes(include=np.number).columns)

  print('cat_col_names: {}'.format(cat_col_names))
  print('num_col_names: {}'.format(num_col_names))
  return cat_col_names,num_col_names

cat_col_names,num_col_names = get_col_names(sdf)

good_columns = cat_col_names + num_col_names

sdf = sdf[good_columns]

for x in cat_col_names:
    sdf[x].fillna('other',inplace=True)
    sdf[x]=sdf[x].astype(str)
    

for y in num_col_names:
    sdf[y].fillna(0,inplace=True)

cat_col_names: ['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management', 'scheme_name', 'extraction_type', 'management', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'waterpoint_type']
num_col_names: ['id', 'gps_height', 'region_code', 'district_code', 'population', 'public_meeting', 'permit']


In [206]:
part_1 = sdf.sample(frac = 0.5, random_state=42)
part_2 = sdf.drop(part_1.index)

In [207]:
X1 = part_1.to_numpy()
X2 = part_2.to_numpy()

In [12]:
sdf

,funder,installer,wpt_name,basin,subvillage,region,lga,ward,scheme_management,scheme_name,extraction_type,management,payment,water_quality,quality_group,quantity,source,waterpoint_type,id,gps_height,region_code,district_code,population,public_meeting,permit
0,Dmdd,DMDD,Dinamu Secondary School,Internal,Magoma,Manyara,Mbulu,Bashay,Parastatal,other,other,parastatal,never pay,soft,good,seasonal,rainwater harvesting,other,50785,1996,21,3,321,1.0,1.0
1,Government Of Tanzania,DWE,Kimnyak,Pangani,Kimnyak,Arusha,Arusha Rural,Kimnyaki,VWC,TPRI pipe line,gravity,vwc,never pay,soft,good,insufficient,spring,communal standpipe,51630,1569,2,2,300,1.0,1.0
2,other,other,Puma Secondary,Internal,Msatu,Singida,Singida Rural,Puma,VWC,P,other,vwc,never pay,soft,good,insufficient,rainwater harvesting,other,17168,1567,13,2,500,1.0,0.0
3,Finn Water,FINN WATER,Kwa Mzee Pange,Ruvuma / Southern Coast,Kipindimbi,Lindi,Liwale,Mkutano,VWC,other,other,vwc,unknown,soft,good,dry,shallow well,other,45559,267,80,43,250,0.0,1.0
4,Bruder,BRUDER,Kwa Mzee Turuka,Ruvuma / Southern Coast,Losonga,Ruvuma,Mbinga,Mbinga Urban,Water Board,BRUDER,gravity,water board,pay monthly,soft,good,enough,spring,communal standpipe,49871,1260,10,3,60,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Danida,Da,Kwambwezi,Wami / Ruvu,Yombo,Pwani,Bagamoyo,Yombo,VWC,Bagamoyo wate,mono,vwc,never pay,soft,good,enough,river,communal standpipe,39307,34,6,1,20,1.0,1.0
14846,Hiap,HIAP,Bonde La Mkondoa,Pangani,Mkondoa,Tanga,Kilindi,Mvungwe,VWC,other,nira/tanira,vwc,pay annually,salty,salty,insufficient,shallow well,hand pump,18990,0,4,7,2960,1.0,0.0
14847,other,other,Bwawani,Internal,Juhudi,Singida,Singida Rural,Ughandi,VWC,other,gravity,vwc,never pay,soft,good,insufficient,dam,communal standpipe,28749,1476,13,2,200,1.0,0.0
14848,Germany,DWE,Kwa John,Lake Nyasa,Namakinga B,Ruvuma,Songea Rural,Maposeni,VWC,Mradi wa maji wa maposeni,gravity,vwc,never pay,soft,good,insufficient,river,communal standpipe,33492,998,10,2,150,1.0,1.0


In [13]:
sdf.describe(include=[np.number]).transpose()


,count,mean,std,min,25%,50%,75%,max
id,14850.0,37161.972929,21359.364833,10.0,18727.0,37361.5,55799.75,74249.0
gps_height,14850.0,655.147609,691.261185,-57.0,0.0,344.0,1308.00,2777.0
region_code,14850.0,15.139057,17.191329,1.0,5.0,12.0,17.00,99.0
district_code,14850.0,5.626397,9.673842,0.0,2.0,3.0,5.00,80.0
population,14850.0,184.114209,469.499332,0.0,0.0,20.0,220.00,11469.0
public_meeting,14850.0,0.857778,0.349290,0.0,1.0,1.0,1.00,1.0
permit,14850.0,0.656835,0.474782,0.0,0.0,1.0,1.00,1.0


In [14]:
sdf.describe(include=[np.object]).transpose()


,count,unique,top,freq
funder,14850,981,Government Of Tanzania,2215
installer,14850,1092,DWE,4349
wpt_name,14850,10840,none,877
basin,14850,9,Lake Victoria,2623
subvillage,14850,8444,Shuleni,140
region,14850,21,Shinyanga,1311
lga,14850,125,Njombe,625
ward,14850,1959,Igosi,79
scheme_management,14850,12,VWC,9124
scheme_name,14850,1790,other,7092


In [16]:
from sklearn.preprocessing import StandardScaler

def scale_values(sdf, num_col_names, cat_col_names):
  X_num = sdf[num_col_names].to_numpy()
  X_cat = sdf[cat_col_names].to_numpy()

  # Be sure to scale all numeric features
  scaler = StandardScaler()
  X_num = scaler.fit_transform(X_num)

  X = np.concatenate((X_num, X_cat), axis=1)
  return X

#X1 = scale_values(part_1, num_col_names, cat_col_names)
#X2 = scale_values(part_2, num_col_names, cat_col_names)
X3 = scale_values(sdf,num_col_names, cat_col_names)

In [221]:
from scipy.spatial import distance


def _validate_vector(u, dtype=None):
    # XXX Is order='c' really necessary?
    u = np.asarray(u, dtype=dtype, order='c').squeeze()
    # Ensure values such as u=1 and u=[1] still return 1-D arrays.
    u = np.atleast_1d(u)
    if u.ndim > 1:
        raise ValueError("Input vector should be 1-D.")
    return u


def danny_dist(u, v, p=2, w=None):
  
  u = _validate_vector(u)
  v = _validate_vector(v)
  if p < 1:
      raise ValueError("p must be at least 1")
  print(u)
  print(v)
  u_v = u - v
  if w is not None:
      w = _validate_weights(w)
      if p == 1:
          root_w = w
      if p == 2:
          # better precision and speed
          root_w = np.sqrt(w)
      else:
          root_w = np.power(w, 1/p)
      u_v = root_w * u_v
  dist = norm(u_v, ord=p)
  return dist


def custom_dist(x1,x2,cat_cols=None,num_cols=None):
  n = len(x1)
  #print(num_cols)
  #dist_num = danny_dist(x1[num_cols],x2[num_cols])
  dist_num = distance.euclidean(x1[num_cols],x2[num_cols])
  dist_cat = distance.hamming(x1[cat_cols],x2[cat_cols])
  dist=(dist_num*sum(num_cols) + dist_cat*sum(cat_cols)) / n
  return dist

In [17]:
from scipy.spatial.distance import pdist


def discover_wrong_values(X):
  cat_cols = [type(x)==str for x in X[1,:]]
  num_cols = [not x for x in cat_cols]

  count=-1
  errors = []
  cat_value=[]
  cat_count=-1
  for y in cat_cols:
    if(y == True):
      cat_value.append(cat_count)
    cat_count+=1


  for x in num_cols:
    count+=1
    if(x == True):
      errors.append(count)
      #print (x, count)
  return errors,cat_value,cat_cols,num_cols

errors,cat_value,cat_cols,num_cols = discover_wrong_values(X3)

In [18]:
#print(sdf.head())
index_col = 0
list_cat_cols = []
for col in sdf.columns:
    print(col,index_col,sdf[col][0])
    list_cat_cols.append(index_col)
    #if(col not in num_col_names):
    #    list_cat_cols.append(index_col)
    index_col+=1
print(list_cat_cols)

funder 0 Dmdd
installer 1 DMDD
wpt_name 2 Dinamu Secondary School
basin 3 Internal
subvillage 4 Magoma
region 5 Manyara
lga 6 Mbulu
ward 7 Bashay
scheme_management 8 Parastatal
scheme_name 9 other
extraction_type 10 other
management 11 parastatal
payment 12 never pay
water_quality 13 soft
quality_group 14 good
quantity 15 seasonal
source 16 rainwater harvesting
waterpoint_type 17 other
id 18 50785
gps_height 19 1996
region_code 20 21
district_code 21 3
population 22 321
public_meeting 23 1.0
permit 24 1.0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [19]:
X3[1]

array([0.6773850415237074, 1.3220518969555621, -0.7643097287376379,
       -0.3748789127208279, 0.24683675876841046, 0.40718927884743017,
       0.7228086380573759, 'Government Of Tanzania', 'DWE', 'Kimnyak',
       'Pangani', 'Kimnyak', 'Arusha', 'Arusha Rural', 'Kimnyaki', 'VWC',
       'TPRI pipe line', 'gravity', 'vwc', 'never pay', 'soft', 'good',
       'insufficient', 'spring', 'communal standpipe'], dtype=object)

In [20]:
from kmodes.kprototypes import KPrototypes
kproto = KPrototypes(n_clusters=10, verbose=2, max_iter=20)
clusters = kproto.fit_predict(X3, categorical=[5,6, 7,8,9, 10, 11, 12,13, 14,15,16,17, 18, 19, 20, 21, 22, 23, 24])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 6660, ncost: 102527.44543175184
Run: 1, iteration: 2/20, moves: 2830, ncost: 100370.3545672659
Run: 1, iteration: 3/20, moves: 869, ncost: 99860.30599276439
Run: 1, iteration: 4/20, moves: 344, ncost: 99765.67518693133
Run: 1, iteration: 5/20, moves: 198, ncost: 99743.46533804703
Run: 1, iteration: 6/20, moves: 114, ncost: 99735.47397031907
Run: 1, iteration: 7/20, moves: 73, ncost: 99728.10616155904
Run: 1, iteration: 8/20, moves: 68, ncost: 99711.82284726987
Run: 1, iteration: 9/20, moves: 72, ncost: 99703.76254725578
Run: 1, iteration: 10/20, moves: 56, ncost: 99699.33790423025
Run: 1, iteration: 11/20, moves: 32, ncost: 99698.18821040458
Run: 1, iteration: 12/20, moves: 21, ncost: 99697.48215443356
Run: 1, iteration: 13/20, moves: 16, ncost: 99696.84879045529
Run: 1, iteration: 14/20, moves: 20, ncost: 99695.11322057032
Run: 1, iteration: 15/20, moves: 14, ncost: 99694.82

Run: 8, iteration: 8/20, moves: 578, ncost: 100960.7928104995
Run: 8, iteration: 9/20, moves: 395, ncost: 100910.11789569113
Run: 8, iteration: 10/20, moves: 292, ncost: 100860.03284064107
Run: 8, iteration: 11/20, moves: 226, ncost: 100830.751394378
Run: 8, iteration: 12/20, moves: 188, ncost: 100812.13174387591
Run: 8, iteration: 13/20, moves: 153, ncost: 100800.00010046364
Run: 8, iteration: 14/20, moves: 131, ncost: 100782.5089908092
Run: 8, iteration: 15/20, moves: 103, ncost: 100770.67367648675
Run: 8, iteration: 16/20, moves: 85, ncost: 100763.68865146759
Run: 8, iteration: 17/20, moves: 67, ncost: 100760.49197913325
Run: 8, iteration: 18/20, moves: 87, ncost: 100743.72437995342
Run: 8, iteration: 19/20, moves: 36, ncost: 100743.16342263593
Run: 8, iteration: 20/20, moves: 23, ncost: 100742.96465668804
Run: 8, iteration: 21/20, moves: 18, ncost: 100742.89531537809
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
St

In [ ]:

#num_cols = [(not x and type(x) == int) for x in cat_cols]

#Need to remove the num columns that are causing issues
num_cols[13] = False
num_cols[15] = False
num_cols[16] = False
#print(num_cols)

#Y1 = pdist(X1, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

In [ ]:
#Y2 = pdist(X2, custom_dist, cat_cols=cat_cols, num_cols=num_cols)

In [21]:
len(clusters)

14850

In [22]:
df['clusters']=clusters

In [23]:
def replace_with_min_year (df, cluster_count, cluster_colname):
  for i in range(cluster_count):
    current_cluster = df[df[cluster_colname] == i]
    current_cluster_with_construction_year = current_cluster[current_cluster['construction_year'] > 0]
    if len(current_cluster_with_construction_year) > 0:
      min_year = current_cluster_with_construction_year['construction_year'].min()
      df.loc[(df[cluster_colname] == i) & (df['construction_year'] == 0), 'construction_year'] = min_year
    else:
      print('no year in this cluster', i)
      #return 1


replace_with_min_year(df,10,'clusters')


In [24]:
df['construction_year'].describe()

count    14850.000000
mean      1984.224781
std         20.133599
min       1960.000000
25%       1960.000000
50%       1986.000000
75%       2004.000000
max       2013.000000
Name: construction_year, dtype: float64

In [267]:
df.to_csv('test_construct_years.csv', encoding='utf-8', index=False)

In [266]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,clusters
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,6
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,6
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,3
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,1960,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,0
